<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/intro-Machine-Learning/blob/main/classes/class_25_desempa%C3%B1o_clasificador/class_25_medidas_desempeno_clasificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table> 

# Medidas de desempeño de un Clasificador y clasificación multiclase: Clase 26 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from sklearn.datasets import fetch_openml


In [2]:
%%time 
mnist = fetch_openml('mnist_784', version = 1, as_frame = False) 

Wall time: 1min 1s


In [3]:
X,y = mnist['data'], mnist['target'] 

# Conversión de `str` a `int64`

In [4]:
y = y.astype(np.int64)

In [ ]:
X_train, X_test, y_train, y_test = X[:60_000], X[60_000:], y[:60_000], y[60_000:] 

In [ ]:
y_train_5 = (y_train==5)
y_test_5 = (y_test==5)

In [ ]:
y[0]

In [ ]:
from sklearn.linear_model import SGDClassifier 

In [ ]:
sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)

In [ ]:

%%time
sgd_clf.fit(X_train, y_train_5) 

In [ ]:
sgd_clf.predict(X_train[:5])

# La validación cruzada con la medida de desempeño de la exactitud 

In [ ]:
%%time 

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_5[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score 

In [ ]:
%%time 

puntajes = cross_val_score(sgd_clf, X_train, y_train_5, cv = 10, scoring = 'accuracy') 

In [ ]:
puntajes 

Nota:  

`shuffle=True` se omitió por error en versiones anteriores del libro.

In [ ]:
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
%%time 

never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")

In [ ]:
from sklearn.metrics import precision_score, recall_score  

In [ ]:
 precision_score(y_train_5, y_predict2) 

In [ ]:
recall_score(y_train_5, y_predict2)

Now your 5-detector does not look as shiny as it did when you looked at its accuracy. 

When it claims an image represents a 5, it is correct only 72.9% of the time. Moreover, it only detects 75.6% of the 5s.

It is often convenient to combine precision and recall into a single metric called the $F_{1}$ score, in particular if you need a simple way to compare two classifiers. 

The $F_{1}$ score is the harmonic mean of precision and recall (Equation 3-3). 

Whereas the regular mean treats all values equally, the harmonic mean gives much more weight to low values. 

As a result, the classifier will only get a high $F_{1}$ score if both recall and precision are high.

$$ F_{1} = \frac{2}{\frac{1}{\text{precision}} + \frac{1}{\text{recall}} } = 2 \cdot \frac{}{} =   $$

To compute the $F_{1}$ score, simply call the `f1_score()` function:

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)

The F score favors classifiers that have similar precision and recall. 

This is not always what you want: in some contexts you mostly care about precision, and in other contexts you really care about recall. 

For example, if you trained a classifier to detect videos that are safe for kids, you would
probably prefer a classifier that rejects many good videos (low recall) but keeps only safe ones (high precision), rather than a classifier that has a much higher recall but lets a few really bad videos show up in your product (in such cases, you may even want to add a human pipeline to check the classifier’s video selection). 

On the other hand, suppose you train a classifier to detect shoplifters in surveillance images: it is probably fine if your classifier has only 30% precision as long as it has 99% recall (sure, the security guards will get a few false alerts, but almost all shoplifters will get
caught).

Unfortunately, you can’t have it both ways: increasing precision reduces
recall, and vice versa. This is called the precision/recall trade-off.

## Precision/Recall Trade-off

To understand this trade-off, let’s look at how the SGDClassifier makes its classification decisions. 

For each instance, it computes a score based on a decision function. 

If that score is greater than a threshold, it assigns the instance to the positive class; otherwise it assigns it to the negative class.

Figure 3-3 shows a few digits positioned from the lowest score on the left to the highest score on the right. 

Suppose the decision threshold is positioned at the central arrow (between the two 5s): you will find 4 true positives (actual 5s) on the right of that threshold, and 1 false positive (actually a 6). 

Therefore, with that threshold, the precision is 80% (4 out of 5). But out of
6 actual 5s, the classifier only detects 4, so the recall is 67% (4 out of 6). If
you raise the threshold (move it to the arrow on the right), the false positive
(the 6) becomes a true negative, thereby increasing the precision (up to
100% in this case), but one true positive becomes a false negative,
decreasing recall down to 50%. 

Conversely, lowering the threshold increases recall and reduces precision.